# part 1


In [59]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import requests # Library for web scraping

print('Libraries imported.')

Libraries imported.


In [60]:
# To run this, you can install BeautifulSoup
# https://pypi.python.org/pypi/beautifulsoup4

# Or download the file
# http://beautiful-soup-4
# and unzip it in the same directory as this file
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import ssl
import csv

print('BeautifulSoup  & csv imported.')

BeautifulSoup  & csv imported.


In [61]:
# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

print('SSL certificate errors ignored.')

SSL certificate errors ignored.


In [62]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(source, 'lxml')

#print(soup.prettify())
print('soup ready')

soup ready


In [63]:
table = soup.find('table',{'class':'wikitable sortable'})
#table

In [64]:
table_rows = table.find_all('tr')

#table_rows

In [65]:
data = []
for row in table_rows:
    data.append([t.text.strip() for t in row.find_all('td')])

df = pd.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighbourhood'])
df = df[~df['PostalCode'].isnull()]  # to filter out bad rows

#print(df.head(5))
#print('***')
#print(df.tail(5))

In [66]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 180 entries, 1 to 180
Data columns (total 3 columns):
PostalCode       180 non-null object
Borough          180 non-null object
Neighbourhood    180 non-null object
dtypes: object(3)
memory usage: 5.6+ KB


In [67]:
df.shape

(180, 3)

In [68]:
import pandas
import requests
from bs4 import BeautifulSoup
website_text = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(website_text,'lxml')

table = soup.find('table',{'class':'wikitable sortable'})
table_rows = table.find_all('tr')

data = []
for row in table_rows:
    data.append([t.text.strip() for t in row.find_all('td')])

df = pandas.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighbourhood'])
df = df[~df['PostalCode'].isnull()]  # to filter out bad rows

#df.head(15)

In [69]:
df.drop(df[df['Borough']=="Not assigned"].index,axis=0, inplace=True)
#df.head()

In [70]:
df1 = df.reset_index()
#df1.head()

In [71]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 4 columns):
index            103 non-null int64
PostalCode       103 non-null object
Borough          103 non-null object
Neighbourhood    103 non-null object
dtypes: int64(1), object(3)
memory usage: 3.3+ KB


In [72]:
df1.shape

(103, 4)

In [73]:
df2= df1.groupby('PostalCode').agg(lambda x: ','.join(x))

#df2.head()

In [74]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 103 entries, M1B to M9W
Data columns (total 2 columns):
Borough          103 non-null object
Neighbourhood    103 non-null object
dtypes: object(2)
memory usage: 2.4+ KB


In [75]:
df2.shape

(103, 2)

In [76]:
df2.loc[df2['Neighbourhood']=="Not assigned",'Neighbourhood']=df2.loc[df2['Neighbourhood']=="Not assigned",'Borough']

#df2.head()

In [77]:
df3 = df2.reset_index()
#df3.head()

In [78]:
df3['Borough']= df3['Borough'].str.replace('nan|[{}\s]','').str.split(',').apply(set).str.join(',').str.strip(',').str.replace(",{2,}",",")

In [79]:
df3.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [80]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 3 columns):
PostalCode       103 non-null object
Borough          103 non-null object
Neighbourhood    103 non-null object
dtypes: object(3)
memory usage: 2.5+ KB


In [81]:
df3.shape

(103, 3)

# part 2


In [82]:
import sys
!{sys.executable} -m pip install geopy

In [83]:
from  geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="my-application")
city ="London"
country ="Uk"
loc = geolocator.geocode(city+','+ country)
print("latitude is :-" ,loc.latitude,"\nlongtitude is:-" ,loc.longitude)

latitude is :- 51.5073219 
longtitude is:- -0.1276474


In [84]:
from  geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode("Toronto, North York, Parkwoods")

print(location.address)
print('')
print((location.latitude, location.longitude))
print('')
print(location.raw)

Parkwoods Village Drive, Parkway East, Don Valley East, North York, Toronto, Golden Horseshoe, Ontario, M3A 2X2, Canada

(43.7587999, -79.3201966)

{'place_id': 124974741, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'way', 'osm_id': 160406961, 'boundingbox': ['43.7576231', '43.761106', '-79.3239088', '-79.316215'], 'lat': '43.7587999', 'lon': '-79.3201966', 'display_name': 'Parkwoods Village Drive, Parkway East, Don Valley East, North York, Toronto, Golden Horseshoe, Ontario, M3A 2X2, Canada', 'class': 'highway', 'type': 'secondary', 'importance': 0.51}


In [85]:
import pandas as pd
#df3.head()

In [86]:
import pandas as pd
df_geopy = pd.DataFrame({'PostalCode': ['M3A', 'M4A', 'M5A'],
                         'Borough': ['North York', 'North York', 'Downtown Toronto'],
                         'Neighbourhood': ['Parkwoods', 'Victoria Village', 'Harbourfront'],})

from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="my-application")

In [87]:
df_geopy1 = df3
#df_geopy1

In [88]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="my-application")

df_geopy1['address'] = df3[['PostalCode', 'Borough', 'Neighbourhood']].apply(lambda x: ', '.join(x), axis=1 )
df_geopy1.head()

,PostalCode,Borough,Neighbourhood,address
0,M1B,Scarborough,Malvern / Rouge,"M1B, Scarborough, Malvern / Rouge"
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,"M1C, Scarborough, Rouge Hill / Port Union / Hi..."
2,M1E,Scarborough,Guildwood / Morningside / West Hill,"M1E, Scarborough, Guildwood / Morningside / We..."
3,M1G,Scarborough,Woburn,"M1G, Scarborough, Woburn"
4,M1H,Scarborough,Cedarbrae,"M1H, Scarborough, Cedarbrae"


In [89]:
df_geopy1 = df3

In [90]:
df_geopy1.shape

(103, 4)

In [91]:
df_geopy1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 4 columns):
PostalCode       103 non-null object
Borough          103 non-null object
Neighbourhood    103 non-null object
address          103 non-null object
dtypes: object(4)
memory usage: 3.3+ KB


In [92]:
df_geopy1.drop(df_geopy1[df_geopy1['Borough']=="Notassigned"].index,axis=0, inplace=True)
#df_geopy1
# code holds true up until i=102
df_geopy1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103 entries, 0 to 102
Data columns (total 4 columns):
PostalCode       103 non-null object
Borough          103 non-null object
Neighbourhood    103 non-null object
address          103 non-null object
dtypes: object(4)
memory usage: 4.0+ KB


In [93]:
#df_geopy1.head()

In [94]:
df_geopy1.shape

(103, 4)

In [95]:
df_geopy1.to_csv('geopy1.csv')
# no data for location after row 75

In [96]:
from  geopy.geocoders import Nominatim 
geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode("M1G, Scarborough, Woburn")


#print(location.address)

#print((location.latitude, location.longitude))

#print(location.raw)

In [97]:
import sys
!{sys.executable} -m pip install geocoder

     |████████████████████████████████| 102kB 7.7MB/s ta 0:00:011


In [98]:
df3.to_csv('geopy.csv')

In [99]:
import csv

with open('geopy.csv') as csvfile:
     reader = csv.DictReader(csvfile)
     #for row in reader:
         #print(row['PostalCode'],row['Borough'], row['Neighbourhood'] )

In [100]:
from  geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode("M1B Scarborough Rouge,Malvern")

#print(location.address)

#print((location.latitude, location.longitude))

#print(location.raw)

In [101]:
from  geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode("Toronto, Highland Creek")

#print(location.address)

#print((location.latitude, location.longitude))

#print(location.raw)

#M1C Scarborough Highland Creek,Rouge Hill,Port Union = no address

In [102]:
from  geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode("Toronto, Morningside")

#print(location.address)

#print((location.latitude, location.longitude))

#print(location.raw)

#M1E Scarborough Guildwood,Morningside,West Hill = no address

In [103]:
import numpy as np
import csv


PostalCode = None
Borough = None
Neighbourhood = None
latData = None
longData = None

LAT_Woburn = 43.7598243
LONG_Woburn = -79.2252908
LAT_Malvern = 43.8091955
LONG_Malvern = -79.2217008
LAT_Highland_Creek = 43.7901172
LONG_Highland_Creek = -79.1733344
LAT_Morningside = 43.7826012
LONG_Morningside = -79.2049579

 
PostalCode = np.array(['M1H','M1B','M1C','M1G '])
Borough = np.array(['Scarborough','Scarborough','Scarborough','Scarborough'])
Neighbourhood = np.array(['Woburn','Malvern','Highland_Creek','Morningside'])
latData = np.array([43.7598243,43.8091955, 43.7901172 , 43.7826012])
longData = np.array([-79.2252908,-79.2217008,-79.1733344, -79.2049579 ])

with open('data.csv', 'w') as file:
    writer = csv.writer(file, delimiter=',')
    writer.writerow('ABXYZ')
    for a,b,x,y,z in np.nditer([ PostalCode.T, Borough.T, Neighbourhood.T, latData.T, longData.T], order='C'):
        writer.writerow([a,b,x,y,z])    

In [104]:
import csv

with open('data.csv') as csvfile:
     reader = csv.DictReader(csvfile)
     for row in reader:
         print(row['A'], row['B'],row['X'], row['Y'], row['Z'])

M1H Scarborough Woburn 43.7598243 -79.2252908
M1B Scarborough Malvern 43.8091955 -79.2217008
M1C Scarborough Highland_Creek 43.7901172 -79.1733344
M1G  Scarborough Morningside 43.7826012 -79.2049579


In [105]:
pd.read_csv('data.csv') 

,A,B,X,Y,Z
0,M1H,Scarborough,Woburn,43.759824,-79.225291
1,M1B,Scarborough,Malvern,43.809196,-79.221701
2,M1C,Scarborough,Highland_Creek,43.790117,-79.173334
3,M1G,Scarborough,Morningside,43.782601,-79.204958


In [106]:
import pandas, os
#os.listdir()

In [107]:
df_geopy=df3
#df_geopy.head()

In [108]:
import geopy
#dir(geopy)

In [109]:
type(df_geopy)

pandas.core.frame.DataFrame

In [110]:
df_geopy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103 entries, 0 to 102
Data columns (total 4 columns):
PostalCode       103 non-null object
Borough          103 non-null object
Neighbourhood    103 non-null object
address          103 non-null object
dtypes: object(4)
memory usage: 4.0+ KB


In [111]:
import sys
!{sys.executable} -m pip install geopy


In [112]:
from geopy.geocoders import Nominatim()
print('Nominatim imported')

SyntaxError: invalid syntax (<ipython-input-112-651592e2bfd6>, line 1)

In [ ]:
df_geopy['address']=df_geopy['PostalCode'] + ',' + df_geopy['Borough'] + ','+ df_geopy['Neighbourhood']
df_geopy.head()

In [ ]:
nom = Nominatim(user_agent="my-application")

In [ ]:
n=nom.geocode('M1B, Scarborough, Rouge,Malvern')
n

In [ ]:
n.latitude

In [ ]:
type(n)

In [ ]:
n2=nom.geocode('M1E Scarborough Guildwood,Morningside,West Hill')
print(n2)

In [ ]:
df_geopy['Coordinates'] =df_geopy['address'].apply(nom.geocode)
df_geopy.head()

In [ ]:
df_geopy.info()

In [ ]:
df_geopy.Coordinates[0]

In [ ]:
print(df_geopy.Coordinates[1])

In [ ]:
df_geopy['latitude']=df_geopy['Coordinates'].apply(lambda x: x.latitude if x !=None else None)
df_geopy['longitude']=df_geopy['Coordinates'].apply(lambda x: x.longitude if x !=None else None)
df_geopy.head()

In [ ]:
df_geopy.to_csv('geo_loc_py.csv')

In [ ]:
print('The latitude of', df_geopy.address[0],  'is', df_geopy.latitude[0], 'and its longitude is',df_geopy.longitude[0])

In [113]:
# Load the Pandas libraries with alias 'pd' 
import pandas as pd 
# Read data from file 'filename.csv' 
# (in the same directory that your python process is based)
# Control delimiters, rows, column names with read_csv (see later) 
data2 = pd.read_csv("geopy.csv") 
# Preview the first 5 lines of the loaded data 
data2.head()

,Unnamed: 0,PostalCode,Borough,Neighbourhood,address
0,0,M1B,Scarborough,Malvern / Rouge,"M1B, Scarborough, Malvern / Rouge"
1,1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,"M1C, Scarborough, Rouge Hill / Port Union / Hi..."
2,2,M1E,Scarborough,Guildwood / Morningside / West Hill,"M1E, Scarborough, Guildwood / Morningside / We..."
3,3,M1G,Scarborough,Woburn,"M1G, Scarborough, Woburn"
4,4,M1H,Scarborough,Cedarbrae,"M1H, Scarborough, Cedarbrae"


In [114]:
!wget -q -O "toronto_coordinates.csv" http://cocl.us/Geospatial_data
print('Coordinates downloaded!')
data3 = pd.read_csv('toronto_coordinates.csv')


Coordinates downloaded!


In [115]:
data3.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
#data3.head()

In [116]:
data1 = pd.merge(data3, data2, how='inner', on=None, left_on=None, right_on=None,
         left_index=False, right_index=False, sort=True,
         suffixes=('_x', '_y'), copy=True, indicator=False,
         validate=None)

data1.head()

,PostalCode,Latitude,Longitude,Unnamed: 0,Borough,Neighbourhood,address
0,M1B,43.806686,-79.194353,0,Scarborough,Malvern / Rouge,"M1B, Scarborough, Malvern / Rouge"
1,M1C,43.784535,-79.160497,1,Scarborough,Rouge Hill / Port Union / Highland Creek,"M1C, Scarborough, Rouge Hill / Port Union / Hi..."
2,M1E,43.763573,-79.188711,2,Scarborough,Guildwood / Morningside / West Hill,"M1E, Scarborough, Guildwood / Morningside / We..."
3,M1G,43.770992,-79.216917,3,Scarborough,Woburn,"M1G, Scarborough, Woburn"
4,M1H,43.773136,-79.239476,4,Scarborough,Cedarbrae,"M1H, Scarborough, Cedarbrae"


In [117]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103 entries, 0 to 102
Data columns (total 7 columns):
PostalCode       103 non-null object
Latitude         103 non-null float64
Longitude        103 non-null float64
Unnamed: 0       103 non-null int64
Borough          103 non-null object
Neighbourhood    103 non-null object
address          103 non-null object
dtypes: float64(2), int64(1), object(4)
memory usage: 6.4+ KB


In [118]:
cols = data1.columns.tolist()
cols

['PostalCode',
 'Latitude',
 'Longitude',
 'Unnamed: 0',
 'Borough',
 'Neighbourhood',
 'address']

In [119]:
new_column_order = ['PostalCode',
 'Borough',
 'Neighbourhood',
 'Latitude',
 'Longitude']
new_column_order

['PostalCode', 'Borough', 'Neighbourhood', 'Latitude', 'Longitude']

In [120]:
data1 = data1[new_column_order]
#data1.head()

In [121]:
sorted_df = data1.sort_values([ 'Neighbourhood', 'Latitude'], ascending=[True, True])
#sorted_df.head()
# no idea how to get it exacly like the exqample :(

In [122]:
sorted_df.reset_index(inplace=True)
#sorted_df.head()

In [123]:
sorted_cols =sorted_df.columns.tolist()
#sorted_cols

In [124]:
new_column_order2 = ['PostalCode',
 'Borough',
 'Neighbourhood',
 'Latitude',
 'Longitude']
new_column_order2

['PostalCode', 'Borough', 'Neighbourhood', 'Latitude', 'Longitude']

In [125]:
sorted_dataframe = sorted_df[new_column_order]
sorted_dataframe.head(12)

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1S,Scarborough,Agincourt,43.794200,-79.262029
1,M8W,Etobicoke,Alderwood / Long Branch,43.602414,-79.543484
2,M3H,NorthYork,Bathurst Manor / Wilson Heights / Downsview North,43.754328,-79.442259
3,M2K,NorthYork,Bayview Village,43.786947,-79.385975
4,M5M,NorthYork,Bedford Park / Lawrence Manor East,43.733283,-79.419750
5,M5E,DowntownToronto,Berczy Park,43.644771,-79.373306
6,M1N,Scarborough,Birch Cliff / Cliffside West,43.692657,-79.264848
7,M6K,WestToronto,Brockton / Parkdale Village / Exhibition Place,43.636847,-79.428191
8,M7Y,EastToronto,Business reply mail Processing CentrE,43.662744,-79.321558
9,M5V,DowntownToronto,CN Tower / King and Spadina / Railway Lands / ...,43.628947,-79.394420


In [126]:
sorted_dataframe.to_csv('sorted_geoloc.csv')

# Part 3


In [14]:
import sys
!{sys.executable} -m pip install folium

     |████████████████████████████████| 92kB 17.0MB/s eta 0:00:01


In [15]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import requests
from pandas.io.json import json_normalize
import json

import requests

from bs4 import BeautifulSoup

from geopy.geocoders import Nominatim

import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

print('Libraries imported!')

Libraries imported!


In [127]:
import pandas as pd

import json

sorted_dataframe.to_json(path_or_buf='geo_toronto.json', orient='table')

In [128]:
with open('geo_toronto.json') as json_data:
    Toronto_data = json.load(json_data)

In [129]:
neighborhoods_data = Toronto_data['data']
neighborhoods_data[0]
#Let's take a look at the first item in this list.

{'index': 0,
 'PostalCode': 'M1S',
 'Borough': 'Scarborough',
 'Neighbourhood': 'Agincourt',
 'Latitude': 43.7942003,
 'Longitude': -79.2620294}

In [130]:
sorted_dataframe.info()
sorted_dataframe.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 5 columns):
PostalCode       103 non-null object
Borough          103 non-null object
Neighbourhood    103 non-null object
Latitude         103 non-null float64
Longitude        103 non-null float64
dtypes: float64(2), object(3)
memory usage: 4.1+ KB


(103, 5)

In [131]:
sorted_dataframe.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1S,Scarborough,Agincourt,43.794200,-79.262029
1,M8W,Etobicoke,Alderwood / Long Branch,43.602414,-79.543484
2,M3H,NorthYork,Bathurst Manor / Wilson Heights / Downsview North,43.754328,-79.442259
3,M2K,NorthYork,Bayview Village,43.786947,-79.385975
4,M5M,NorthYork,Bedford Park / Lawrence Manor East,43.733283,-79.419750


In [132]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(sorted_dataframe['Borough'].unique()),
        sorted_dataframe.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


In [134]:
address = 'Adelaide'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Adelaide are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Adelaide are -34.9281805, 138.5999312.


In [18]:
# open Wiki page with Beautiful Soup
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(data, 'html.parser')

In [20]:
!conda install -c conda-forge folium=0.5.0 

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                       

In [21]:
import pandas as pd
import folium

print('imported pandas & folium')

imported pandas & folium


In [39]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="tl-toronto-neigh")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.6534817, -79.3839347.


In [45]:
!wget -q -O "toronto_coordinates.csv" http://cocl.us/Geospatial_data
print('Coordinates downloaded!')
data3 = pd.read_csv('toronto_coordinates.csv')

Coordinates downloaded!


In [136]:
import pandas as pd
import folium

#grab a random sample from df
subset_of_df = sorted_dataframe.sample(n=11)
map_test = folium.Map(location=[subset_of_df['Latitude'].mean(), 
                                subset_of_df['Longitude'].mean()], 
                      zoom_start=10)
#creating a Marker for each point in df_sample. Each point will get a popup with their zip


    
#map_test

#open map_test.html in browser
map_test.save("map_test.html")

# if you cannot generate the maps open PGA_map_*.html from the zip file

In [137]:
from folium.plugins import MarkerCluster
map_borough = folium.Map(location=[subset_of_df['Latitude'].mean(), 
 subset_of_df['Longitude'].mean()], 
 zoom_start=10)
mc = MarkerCluster()
#creating a Marker for each point in df_sample. Each point will get a popup with their zip
for row in subset_of_df.itertuples():
    mc.add_child(folium.Marker(location=[row.Latitude,  row.Longitude],
                 popup=row.Borough))
    map_borough.add_child(mc)


#map_borough

#open in map_borough.html browser 
map_borough.save("map_borough.html")

#if you cannot generate the maps open PGA_map_*.html from the zip file

ImportError: cannot import name '_validate_location'

In [138]:
import pandas as pd
import folium



#grab a random sample from df
toronto_n = sorted_dataframe.sample(n=20)
map_toronto = folium.Map(location=[toronto_n['Latitude'].mean(), 
                                toronto_n['Longitude'].mean()], 
                      zoom_start=10)
#creating a Marker for each point in df_sample. Each point will get a popup with their zip
for row in toronto_n.itertuples():
    map_toronto.add_child(folium.Marker(location=[row.Latitude ,row.Longitude],
           popup=row.Neighbourhood))

    
map_toronto 

#open map_toronto.html in browser

map_toronto.save("map_toronto20.html")

#if you cannot generate the maps open PGA_map_*.html from the zip file

In [139]:
sorted_dataframe.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1S,Scarborough,Agincourt,43.794200,-79.262029
1,M8W,Etobicoke,Alderwood / Long Branch,43.602414,-79.543484
2,M3H,NorthYork,Bathurst Manor / Wilson Heights / Downsview North,43.754328,-79.442259
3,M2K,NorthYork,Bayview Village,43.786947,-79.385975
4,M5M,NorthYork,Bedford Park / Lawrence Manor East,43.733283,-79.419750


In [140]:
sorted_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 5 columns):
PostalCode       103 non-null object
Borough          103 non-null object
Neighbourhood    103 non-null object
Latitude         103 non-null float64
Longitude        103 non-null float64
dtypes: float64(2), object(3)
memory usage: 4.1+ KB


In [141]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(sorted_dataframe['Borough'].unique()),
        sorted_dataframe.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


In [143]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [144]:
# create map of Toronto using latitude and longitude values
map_toronto_neighbourhoods = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(sorted_dataframe['Latitude'], sorted_dataframe['Longitude'], sorted_dataframe['Borough'], sorted_dataframe['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=2,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_neighbourhoods)  
    
map_toronto_neighbourhoods

map_toronto_neighbourhoods.save("map_toronto_neighbourhoods.html")

#open map_toronto_neighbourhoods.html in browser
#if you cannot generate the maps open PGA_map_*.html from the zip file

In [146]:
address = 'York, Toronto'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of York, Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of York, Toronto are 43.67910515, -79.49118414007154.


In [147]:
york_data = sorted_dataframe[sorted_dataframe['Borough'] == 'York'].reset_index(drop=True)
york_data


,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M6E,York,Caledonia-Fairbanks,43.689026,-79.453512
1,M6M,York,Del Ray / Mount Dennis / Keelsdale and Silvert...,43.691116,-79.476013
2,M6C,York,Humewood-Cedarvale,43.693781,-79.428191
3,M6N,York,Runnymede / The Junction North,43.673185,-79.487262
4,M9N,York,Weston,43.706876,-79.518188


In [148]:
york_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 5 columns):
PostalCode       5 non-null object
Borough          5 non-null object
Neighbourhood    5 non-null object
Latitude         5 non-null float64
Longitude        5 non-null float64
dtypes: float64(2), object(3)
memory usage: 280.0+ bytes


In [149]:
# create map of Manhattan using latitude and longitude values
map_york_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(york_data['Latitude'], york_data['Longitude'], york_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_york_toronto)  
    
map_york_toronto

map_york_toronto.save("map_york_toronto.html")

#open map_york_toronto.html in browser
#if you cannot generate the maps open PGA_map_*.html from the zip file

In [161]:
CLIENT_ID = 'DGDGPSK0SAMMO2DWNGBO2L5YATZZWSW5X0LAQ12SDL1HIKBI' # your Foursquare ID
CLIENT_SECRET = '5KNUJATWBEKT14I03YOCAEBNFSSV3VIW2TPUQI1R0QUP0ATT' # your Foursquare Secret
VERSION = '20200501' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


Your credentails:
CLIENT_ID: DGDGPSK0SAMMO2DWNGBO2L5YATZZWSW5X0LAQ12SDL1HIKBI
CLIENT_SECRET:5KNUJATWBEKT14I03YOCAEBNFSSV3VIW2TPUQI1R0QUP0ATT


In [154]:
york_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 5 columns):
PostalCode       5 non-null object
Borough          5 non-null object
Neighbourhood    5 non-null object
Latitude         5 non-null float64
Longitude        5 non-null float64
dtypes: float64(2), object(3)
memory usage: 280.0+ bytes


In [155]:
neighbourhood_latitude = york_data.loc[0, 'Latitude'] # neighborhood latitude value
neighbourhood_longitude = york_data.loc[0, 'Longitude'] # neighborhood longitude value

neighbourhood_name = york_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))


Latitude and longitude values of Caledonia-Fairbanks are 43.6890256, -79.453512.


In [164]:
LIMIT = 100

radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(

CLIENT_ID, 

CLIENT_SECRET, 

VERSION, 

neighbourhood_latitude, 

neighbourhood_longitude, 

radius, 

LIMIT)

url

'https://api.foursquare.com/v2/venues/explore?&client_id=DGDGPSK0SAMMO2DWNGBO2L5YATZZWSW5X0LAQ12SDL1HIKBI&client_secret=5KNUJATWBEKT14I03YOCAEBNFSSV3VIW2TPUQI1R0QUP0ATT&v=20200501&ll=43.6890256,-79.453512&radius=500&limit=100'

In [165]:
york_results = requests.get(url).json()
#york_results

In [166]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [167]:
york_venues = york_results['response']['groups'][0]['items']
    
york_nearby_venues = json_normalize(york_venues) # flatten JSON

# filter columns
york_filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
york_nearby_venues = york_nearby_venues.loc[:, york_filtered_columns]

# filter the category for each row
york_nearby_venues['venue.categories'] = york_nearby_venues.apply(get_category_type, axis=1)

# clean columns
york_nearby_venues.columns = [col.split(".")[-1] for col in york_nearby_venues.columns]

york_nearby_venues.head()

,name,categories,lat,lng
0,Star Spa,Spa,43.690665,-79.456229
1,Nairn Park,Park,43.690654,-79.456300
2,Maximum Woman,Women's Store,43.690651,-79.456333
3,Fairbank Memorial Park,Park,43.692028,-79.448924


In [168]:
york_nearby_venues.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 4 columns):
name          4 non-null object
categories    4 non-null object
lat           4 non-null float64
lng           4 non-null float64
dtypes: float64(2), object(2)
memory usage: 208.0+ bytes


In [169]:
print('{} venues were returned by Foursquare.'.format(york_nearby_venues.shape[0]))


4 venues were returned by Foursquare.


In [170]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [171]:
york_venues = getNearbyVenues(names=york_data['Neighbourhood'],
                                   latitudes=york_data['Latitude'],
                                   longitudes=york_data['Longitude']
                                  )

Caledonia-Fairbanks
Del Ray / Mount Dennis / Keelsdale and Silverthorn
Humewood-Cedarvale
Runnymede / The Junction North
Weston


In [172]:
york_venues.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Caledonia-Fairbanks,43.689026,-79.453512,Star Spa,43.690665,-79.456229,Spa
1,Caledonia-Fairbanks,43.689026,-79.453512,Nairn Park,43.690654,-79.456300,Park
2,Caledonia-Fairbanks,43.689026,-79.453512,Maximum Woman,43.690651,-79.456333,Women's Store
3,Caledonia-Fairbanks,43.689026,-79.453512,Fairbank Memorial Park,43.692028,-79.448924,Park
4,Del Ray / Mount Dennis / Keelsdale and Silvert...,43.691116,-79.476013,Subway,43.690218,-79.474050,Sandwich Place


In [173]:
york_venues.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 7 columns):
Neighbourhood              16 non-null object
Neighbourhood Latitude     16 non-null float64
Neighbourhood Longitude    16 non-null float64
Venue                      16 non-null object
Venue Latitude             16 non-null float64
Venue Longitude            16 non-null float64
Venue Category             16 non-null object
dtypes: float64(4), object(3)
memory usage: 976.0+ bytes


In [174]:
york_venues.groupby('Neighbourhood').count()


,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Caledonia-Fairbanks,4,4,4,4,4,4
Del Ray / Mount Dennis / Keelsdale and Silverthorn,4,4,4,4,4,4
Humewood-Cedarvale,4,4,4,4,4,4
Runnymede / The Junction North,3,3,3,3,3,3
Weston,1,1,1,1,1,1


In [175]:
print('There are {} uniques categories.'.format(len(york_venues['Venue Category'].unique())))


There are 13 uniques categories.


In [176]:
york_onehot = pd.get_dummies(york_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
york_onehot['Neighbourhood'] = york_venues['Neighbourhood'] 

# move neighborhood column to the first column
york_fixed_columns = [york_onehot.columns[-1]] + list(york_onehot.columns[:-1])
york_onehot = york_onehot[york_fixed_columns]

york_onehot.head()


,Neighbourhood,Brewery,Bus Line,Caribbean Restaurant,Coffee Shop,Dog Run,Fast Food Restaurant,Field,Hockey Arena,Park,Sandwich Place,Spa,Trail,Women's Store
0,Caledonia-Fairbanks,0,0,0,0,0,0,0,0,0,0,1,0,0
1,Caledonia-Fairbanks,0,0,0,0,0,0,0,0,1,0,0,0,0
2,Caledonia-Fairbanks,0,0,0,0,0,0,0,0,0,0,0,0,1
3,Caledonia-Fairbanks,0,0,0,0,0,0,0,0,1,0,0,0,0
4,Del Ray / Mount Dennis / Keelsdale and Silvert...,0,0,0,0,0,0,0,0,0,1,0,0,0


In [177]:
york_onehot.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 14 columns):
Neighbourhood           16 non-null object
Brewery                 16 non-null uint8
Bus Line                16 non-null uint8
Caribbean Restaurant    16 non-null uint8
Coffee Shop             16 non-null uint8
Dog Run                 16 non-null uint8
Fast Food Restaurant    16 non-null uint8
Field                   16 non-null uint8
Hockey Arena            16 non-null uint8
Park                    16 non-null uint8
Sandwich Place          16 non-null uint8
Spa                     16 non-null uint8
Trail                   16 non-null uint8
Women's Store           16 non-null uint8
dtypes: object(1), uint8(13)
memory usage: 416.0+ bytes


In [178]:
york_grouped = york_onehot.groupby('Neighbourhood').mean().reset_index()
york_grouped.head()


,Neighbourhood,Brewery,Bus Line,Caribbean Restaurant,Coffee Shop,Dog Run,Fast Food Restaurant,Field,Hockey Arena,Park,Sandwich Place,Spa,Trail,Women's Store
0,Caledonia-Fairbanks,0.000000,0.000000,0.000000,0.00,0.00,0.0,0.00,0.00,0.5,0.00,0.25,0.00,0.25
1,Del Ray / Mount Dennis / Keelsdale and Silvert...,0.000000,0.000000,0.000000,0.25,0.00,0.5,0.00,0.00,0.0,0.25,0.00,0.00,0.00
2,Humewood-Cedarvale,0.000000,0.000000,0.000000,0.00,0.25,0.0,0.25,0.25,0.0,0.00,0.00,0.25,0.00
3,Runnymede / The Junction North,0.333333,0.333333,0.333333,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.00
4,Weston,0.000000,0.000000,0.000000,0.00,0.00,0.0,0.00,0.00,1.0,0.00,0.00,0.00,0.00


In [179]:
york_grouped.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 14 columns):
Neighbourhood           5 non-null object
Brewery                 5 non-null float64
Bus Line                5 non-null float64
Caribbean Restaurant    5 non-null float64
Coffee Shop             5 non-null float64
Dog Run                 5 non-null float64
Fast Food Restaurant    5 non-null float64
Field                   5 non-null float64
Hockey Arena            5 non-null float64
Park                    5 non-null float64
Sandwich Place          5 non-null float64
Spa                     5 non-null float64
Trail                   5 non-null float64
Women's Store           5 non-null float64
dtypes: float64(13), object(1)
memory usage: 640.0+ bytes


In [180]:
num_top_venues = 3

for hood in york_grouped['Neighbourhood']:
    print("----"+hood+"----")
    york_temp = york_grouped[york_grouped['Neighbourhood'] == hood].T.reset_index()
    york_temp.columns = ['venue','freq']
    york_temp = york_temp.iloc[1:]
    york_temp['freq'] = york_temp['freq'].astype(float)
    york_temp = york_temp.round({'freq': 2})
    print(york_temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Caledonia-Fairbanks----
           venue  freq
0           Park  0.50
1            Spa  0.25
2  Women's Store  0.25


----Del Ray / Mount Dennis / Keelsdale and Silverthorn----
                  venue  freq
0  Fast Food Restaurant  0.50
1           Coffee Shop  0.25
2        Sandwich Place  0.25


----Humewood-Cedarvale----
          venue  freq
0       Dog Run  0.25
1         Field  0.25
2  Hockey Arena  0.25


----Runnymede / The Junction North----
                  venue  freq
0               Brewery  0.33
1              Bus Line  0.33
2  Caribbean Restaurant  0.33


----Weston----
      venue  freq
0      Park   1.0
1   Brewery   0.0
2  Bus Line   0.0




In [183]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [201]:
num_top_venues = 10


indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
york_neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)

york_neighbourhoods_venues_sorted

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


In [203]:
york_neighbourhoods_venues_sorted['Neighbourhood'] = york_grouped['Neighbourhood']

york_neighbourhoods_venues_sorted.head(4)

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Caledonia-Fairbanks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Del Ray / Mount Dennis / Keelsdale and Silvert...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Humewood-Cedarvale,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Runnymede / The Junction North,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [204]:
for ind in np.arange(york_grouped.shape[0]):
    york_neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(york_grouped.iloc[ind,2 :], num_top_venues)

york_neighbourhoods_venues_sorted.head(4)


,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Caledonia-Fairbanks,Park,Women's Store,Spa,Trail,Sandwich Place,Hockey Arena,Field,Fast Food Restaurant,Dog Run,Coffee Shop
1,Del Ray / Mount Dennis / Keelsdale and Silvert...,Fast Food Restaurant,Sandwich Place,Coffee Shop,Women's Store,Trail,Spa,Park,Hockey Arena,Field,Dog Run
2,Humewood-Cedarvale,Trail,Hockey Arena,Field,Dog Run,Women's Store,Spa,Sandwich Place,Park,Fast Food Restaurant,Coffee Shop
3,Runnymede / The Junction North,Caribbean Restaurant,Women's Store,Trail,Spa,Sandwich Place,Park,Hockey Arena,Field,Fast Food Restaurant,Dog Run


In [205]:
# set number of clusters
kclusters = 2

york_grouped_clustering = york_grouped.drop('Neighbourhood', 1)

# run k-means clustering
york_kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(york_grouped_clustering)

# check cluster labels generated for each row in the dataframe
york_kmeans.labels_[0:5] 

array([0, 1, 1, 1, 0], dtype=int32)

In [206]:
# add clustering labels
york_neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', york_kmeans.labels_)

york_merged = york_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
york_merged = york_merged.join(york_neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

york_merged


,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M6E,York,Caledonia-Fairbanks,43.689026,-79.453512,0,Park,Women's Store,Spa,Trail,Sandwich Place,Hockey Arena,Field,Fast Food Restaurant,Dog Run,Coffee Shop
1,M6M,York,Del Ray / Mount Dennis / Keelsdale and Silvert...,43.691116,-79.476013,1,Fast Food Restaurant,Sandwich Place,Coffee Shop,Women's Store,Trail,Spa,Park,Hockey Arena,Field,Dog Run
2,M6C,York,Humewood-Cedarvale,43.693781,-79.428191,1,Trail,Hockey Arena,Field,Dog Run,Women's Store,Spa,Sandwich Place,Park,Fast Food Restaurant,Coffee Shop
3,M6N,York,Runnymede / The Junction North,43.673185,-79.487262,1,Caribbean Restaurant,Women's Store,Trail,Spa,Sandwich Place,Park,Hockey Arena,Field,Fast Food Restaurant,Dog Run
4,M9N,York,Weston,43.706876,-79.518188,0,Park,Women's Store,Trail,Spa,Sandwich Place,Hockey Arena,Field,Fast Food Restaurant,Dog Run,Coffee Shop


In [225]:
york_map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(york_merged['Latitude'], york_merged['Longitude'], york_merged['Neighbourhood'], york_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(york_map_clusters)
       
york_map_clusters

york_map_clusters.save("york_map_clusters.html")

york_map_clusters

In [223]:
york_merged.loc[york_merged['Cluster Labels'] == 0, york_merged.columns[[1] + list(range(5, york_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,York,0,Park,Women's Store,Spa,Trail,Sandwich Place,Hockey Arena,Field,Fast Food Restaurant,Dog Run,Coffee Shop
4,York,0,Park,Women's Store,Trail,Spa,Sandwich Place,Hockey Arena,Field,Fast Food Restaurant,Dog Run,Coffee Shop


In [220]:
york_merged.loc[york_merged['Cluster Labels'] == 1, york_merged.columns[[1] + list(range(5, york_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,York,1,Fast Food Restaurant,Sandwich Place,Coffee Shop,Women's Store,Trail,Spa,Park,Hockey Arena,Field,Dog Run
2,York,1,Trail,Hockey Arena,Field,Dog Run,Women's Store,Spa,Sandwich Place,Park,Fast Food Restaurant,Coffee Shop
3,York,1,Caribbean Restaurant,Women's Store,Trail,Spa,Sandwich Place,Park,Hockey Arena,Field,Fast Food Restaurant,Dog Run
